# Inverse Optimization for DFS
_Applying Ghobadi and Mahmoudzadeh 2021_

In [ ]:
using JuMP
using Gurobi
using LinearAlgebra

## Structures and parameters

In [ ]:
struct ForwardParams
    n_paths::Int
    n_commodities::Int

    capacities::Vector{Number}

    design_costs::Vector
    flow_costs::Matrix

    enabled_flows::Matrix{Bool}

    function ForwardParams(; n_paths, n_commodities, capacities, design_costs, flow_costs, enabled_flows=nothing)
        shape = (n_paths, n_commodities)
        
        if isnothing(enabled_flows) 
            enabled_flows = ones(Bool, shape)
        elseif size(enabled_flows) != shape
            error("Invalid shape $(size(enabled_flows)) for `disabled_flows`, should be $(shape)")
        end
        
        new(n_paths, n_commodities, capacities, design_costs, flow_costs, enabled_flows)
    end
end

struct ForwardSolution
    x_sol::Matrix
    z_sol::Vector
end

In [ ]:
function forward_example_params()::ForwardParams
    enabled_flows = ones(Bool, (2, 2))
    enabled_flows[1, 2] = false

    return ForwardParams(
        n_paths=2, 
        n_commodities=2,
        capacities=[100, 100],
        design_costs=[100, 10],
        flow_costs=[10 10 ; 100 100],
        enabled_flows=enabled_flows
    )
end

function forward_example_demand()::Vector
    return [10, 6]
end

function sol_vector(sol::ForwardSolution)::Vector
    flat_xs = reshape(sol.x_sol, (length(sol.x_sol), 1))

    return vec(vcat(flat_xs, sol.z_sol))
end

## Forward problem

In [ ]:
function create_forward_problem(params::ForwardParams, demands::Vector)::Model
    model = Model(Gurobi.Optimizer)

    @variable(model, z[1:params.n_paths], Bin)
    @variable(model, x[1:params.n_paths, 1:params.n_commodities] >= 0)

    @objective(model, Min, params.design_costs' * z + sum(params.flow_costs .* x))

    @constraint(model, [k = 1:params.n_commodities], sum(x[:, k]) .== demands[k])
    @constraint(model, [i = 1:params.n_paths], sum(x[i, :]) <= params.capacities[i] * z[i])
    @constraint(model, [i = 1:params.n_paths, k = 1:params.n_paths; !params.enabled_flows[i, k]], x[i, k] .== 0)

    return model
end

function solve_forward_problem!(model::Model)::ForwardSolution
    optimize!(model)

    x_sol = value.(model[:x])
    z_sol = value.(model[:z])

    return ForwardSolution(x_sol, z_sol)
end

function create_and_solve_forward_problem(params::ForwardParams, demands::Vector)::ForwardSolution
    return solve_forward_problem!(create_forward_problem(params, demands))
end

In [ ]:
forward_model = create_forward_problem(forward_example_params(), forward_example_demand())
forward_sol = solve_forward_problem!(forward_model)

In [ ]:
println(forward_sol.x_sol)
println(forward_sol.z_sol)

## Inverse problem (demand only)

In [ ]:
struct InverseDemandSolution
    demands::Vector
end

In [ ]:
function create_A_demand(n_paths::Integer, n_commodities::Integer, n_variables::Integer, enabled_flows::Matrix{Bool})
    A_pos = zeros(Number, (n_commodities, n_variables))
    for k in 1:n_commodities
        shift_amount = (k - 1) * n_paths
        shifted_range = (1:n_paths) .+ shift_amount
        A_pos[k, shifted_range] .= Int.(params.enabled_flows[:,k])
    end
    A = vcat(A_pos, -A_pos)

    return A
end

function create_Gh_demand(n_paths::Integer, n_commodities::Integer, n_variables::Integer, enabled_flows::Matrix{Bool})
    n_flows = n_paths * n_commodities

    G_paths = zeros(Number, (n_paths, n_variables))
    for p in 1:n_paths
        G_paths[p, p:n_commodities:n_commodities*n_paths] .= Int.(params.enabled_flows[p, :])
        G_paths[p, n_flows + p] = params.capacities[p]
    end
    G_nonneg = diagm(ones(n_variables))
    G_binary = hcat(zeros((n_paths, n_flows)), diagm(ones(n_paths)))
    
    G = vcat(.-G_paths, G_nonneg, G_binary)
    h = zeros(size(G)[1])

    return G,h
end

function create_AGh_demand(params::ForwardParams)
    n_paths, n_commodities = params.n_paths, params.n_commodities
    n_flows = n_paths * n_commodities
    n_variables = n_flows + n_paths

    A = create_A_demand(n_paths, n_commodities, n_variables, params.enabled_flows)
    G,h = create_Gh_demand(n_paths, n_commodities, n_variables, params.enabled_flows)

    return A, G, h
end

function add_half_space_constraint_demand(G::Matrix, h::Vector, solutions::Tuple{ForwardSolution}, params::ForwardParams)
    flat_flow_costs = reshape(params.flow_costs, (length(params.flow_costs), 1))
    full_costs = vcat(flat_flow_costs, params.design_costs)

    optimal_cost = minimum(sol -> full_costs' * sol_vector(sol), solutions)

    G_hs = vcat(G, full_costs')
    h_hs = vcat(h, optimal_cost)

    return G_hs, h_hs
end

In [ ]:
function create_b_variables_demand!(model::Model, n_commodities::Integer)
    @variable(model, b[1:(2 * n_commodities)])
    @constraint(model, [i = 1:n_commodities], b[i] == -b[n_commodities + i])
    
    return b
end

function add_inverse_constraints_demand!(model::Model, solutions::Tuple{ForwardSolution}, A::Matrix, b::Vector, G::Matrix, h::Vector)
    @constraint(model, [sol in solutions], A*sol_vector(sol) .>= b)
end

function add_inverse_objective_demand!(model::Model, A::Matrix, b::Vector)
end

function create_inverse_demand_problem(params::ForwardParams, demand::Vector, solutions::ForwardSolution...)
    model = Model(Gurobi.Optimizer)

    A, G, h = create_AGh_demand(params)
    G, h = add_half_space_constraint_demand(G, h, solutions, params)
    
    b = create_b_variables_demand!(model, params.n_commodities)
    print(typeof(solutions))

    add_inverse_constraints_demand!(model, solutions, A, b, G, h)
    add_inverse_objective_demand!(model, A, b)    

    return model
end

In [ ]:
params = forward_example_params()
demand = forward_example_demand()

forward_model = create_forward_problem(params, demand)
forward_sol = solve_forward_problem!(forward_model)

In [ ]:
inverse_demand_model = create_inverse_demand_problem(params, demand, forward_sol)
print(inverse_demand_model)

In [ ]:
function solve_inverse_demand_problem!(model::Model)::InverseDemandSolution
    optimize!(model)

    b_sol = value.(model[:b])
    half = length(b_sol) ÷ 2
    b_first, b_second = b_sol[1:half], b_sol[half+1:end]
    
    demands = (all(b_first .>= 0)) ? b_first : b_second

    return InverseDemandSolution(demands)
end

In [ ]:
solve_inverse_demand_problem!(inverse_demand_model)

## Inverse problem (linear regression on demand)

In [ ]:
struct InverseLinRegParams
    n_features::Integer
    n_commodities::Integer

    weights::Matrix
    forward_params::ForwardParams

    function InverseLinRegParams(; weights::Matrix, forward_params::ForwardParams)
        return new(size(weights)[2], forward_params.n_commodities, weights, forward_params)
    end
end

struct InverseLinRegSolution
    forward_sol::ForwardSolution
    linreg_features::Vector
end

In [ ]:
function inverse_linreg_example_params()
    weights = [1.5 2; 1 1]

    return InverseLinRegParams(weights=weights, forward_params=forward_example_params())
end

function inverse_linreg_example_featuress()
    return [[4, 2], [3, 1]]
end

In [ ]:
function predict_demand(params::InverseLinRegParams, linreg_features::Vector)
    return params.weights * linreg_features
end

function generate_inverse_linreg_solutions(params::InverseLinRegParams, linreg_featuress)
    demandss = [predict_demand(params, features) for features in linreg_featuress]
    forward_sols = [create_and_solve_forward_problem(params.forward_params, demands) for demands in demandss]

    return [InverseLinRegSolution(forward_sol, linreg_features) for (forward_sol, linreg_features) in zip(forward_sols, linreg_featuress)]
end

In [ ]:
function linreg_sol_vector(sol::InverseLinRegSolution)::Vector
    return vcat(sol_vector(sol.forward_sol), sol.linreg_features)
end

function create_A_weights!(model::Model, n_commodities::Integer, n_features::Integer)
    @variable(model, w[1:n_commodities, 1:n_features])
    return vcat(w, .-w)
end

function normalize_A_rows!(model::Model, A_full::Matrix)
    # Not sure this is actually needed 
end

function create_A_linreg!(model::Model, inv_params::InverseLinRegParams)
    A_demand, _, _ = create_AGh_demand(inv_params.forward_params)
    A_w = create_A_weights!(model, inv_params.n_commodities, inv_params.n_features)

    A = hcat(A_demand, A_w)

    normalize_A_rows!(model, A)
    return A
end

function create_b_linreg!(model::Model, params::InverseLinRegParams)
    return zeros(2*params.n_commodities)
end

In [ ]:
inv_linreg_params = inverse_linreg_example_params()
inv_linreg_model = Model(Gurobi.Optimizer)

A = create_A_linreg!(inv_linreg_model, inv_linreg_params)

In [ ]:
function add_linreg_inverse_constraints!(model::Model, A::Matrix, b::Vector, solutions::Vector{InverseLinRegSolution})
    println("Test $(A)")
    
    for sol in solutions
        sol_vec = linreg_sol_vector(sol)

        # Workaround since 
        for row in 1:size(A)[1]
            a = A[row, :]

            println("$(row) AT $(a)")
            @constraint(model, sol_vec' * a .>= b[row])
        end
    end
end

function add_linreg_inverse_objective!(model::Model, A::Matrix, b::Vector)

end

function create_inverse_linreg_problem(params::InverseLinRegParams, solutions::Vector{InverseLinRegSolution})
    model = Model(Gurobi.Optimizer)

    A = create_A_linreg!(model, params)
    b = create_b_linreg!(model, params)

    add_linreg_inverse_constraints!(model, A, b, solutions)
    add_linreg_inverse_objective!(model, A, b)

    return model
end

In [ ]:
linreg_params = inverse_linreg_example_params()
linreg_featuress = inverse_linreg_example_featuress()
solutions = generate_inverse_linreg_solutions(linreg_params, linreg_featuress)

In [ ]:
linreg_model = create_inverse_linreg_problem(linreg_params, solutions)
print(linreg_model)

In [ ]:
optimize!(linreg_model)
value.(linreg_model[:w])